# doc2vecを用いた単語の特徴ベクトル学習 (テストラン: サブセットデータ)

In [2]:
import gensim
import os
import collections
import random
import csv
import sys

In [3]:
csv.field_size_limit(sys.maxsize)

131072

In [ ]:
corpusfile = 'novel2vec-subset.txt'
modelfile = 'novel2vec-test.model'

In [4]:
def read_corpus():
    with open(corpusfile, newline='', encoding='utf-8') as csvfile:
        novels = csv.reader(csvfile)
        for novel in novels:
            novel_id, tokens_text = novel
            tokens = gensim.utils.to_unicode(tokens_text).split()
            yield gensim.models.doc2vec.TaggedDocument(tokens, [novel_id])

In [5]:
corpus = list(read_corpus())

In [6]:
model = gensim.models.doc2vec.Doc2Vec(size=20, min_count=10, iter=10, workers=16)

In [7]:
model.build_vocab(corpus)

In [8]:
%time model.train(corpus)

CPU times: user 3min 40s, sys: 428 ms, total: 3min 41s
Wall time: 25.7 s


33599150

In [9]:
model.save(modelfile)

In [25]:
ranks = []
second_ranks = []
for doc_id in range(len(corpus)):
    inferred_vector = model.infer_vector(corpus[doc_id].words)
    sims = model.docvecs.most_similar([inferred_vector], topn=len(model.docvecs))
    rank = [docid for docid, sim in sims].index(corpus[doc_id].tags[0])
    ranks.append(rank)
    second_ranks.append(sims[1])

In [28]:
collections.Counter(ranks)

Counter({0: 984, 1: 13, 2: 1, 4: 2})